![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [244]:
-- Write your SQL query here
SELECT * FROM grocery_sales;

,index,Store_ID,Date,Dept,Weekly_Sales
0,0,1,2010-02-05,1,24924.50
1,1,1,2010-02-05,26,11737.12
2,2,1,2010-02-05,17,13223.76
3,3,1,2010-02-05,45,37.44
4,4,1,2010-02-05,28,1085.29
...,...,...,...,...,...
231517,232414,24,2011-05-06,8,49471.07
231518,232415,24,2011-05-06,50,1210.00
231519,232416,24,2011-05-06,87,25893.32
231520,232417,24,2011-05-06,85,1357.83


In [245]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

In [246]:
def transform(raw_data):
    clean_data=raw_data.fillna({'Date':'2018-01-01'})
    clean_data=clean_data.fillna(0)
    clean_data['Month']=clean_data['Date'].dt.month
    clean_data=clean_data[clean_data['Weekly_Sales']>10000]
    keeped_columns=['Store_ID','Month','Dept','IsHoliday','Weekly_Sales','CPI','Unemployment']
    clean_data=clean_data[keeped_columns]
    # Write your code here
    return clean_data

In [247]:
clean_data=transform(merged_df)
clean_data

,Store_ID,Month,Dept,IsHoliday,Weekly_Sales,CPI,Unemployment
0,1,2,1,0,24924.50,211.096358,8.106
1,1,2,26,0,11737.12,211.096358,8.106
2,1,2,17,0,13223.76,211.096358,8.106
5,1,2,79,0,46729.77,211.096358,0.000
6,1,2,55,0,21249.31,211.096358,0.000
...,...,...,...,...,...,...,...
231513,24,5,40,0,45396.26,134.514367,8.212
231515,24,5,93,0,41295.84,134.514367,8.212
231516,24,5,9,0,24024.18,134.514367,8.212
231517,24,5,8,0,49471.07,134.514367,8.212


In [248]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    # Write your code here
    avg_weeklysales_per_month=clean_data.groupby('Month')['Weekly_Sales'].mean().round()
    avg_weeklysales_per_month=avg_weeklysales_per_month.reset_index()
    avg_weeklysales_per_month=avg_weeklysales_per_month.rename(columns={'Weekly_Sales':'Avg_Sales'})
    return avg_weeklysales_per_month

In [249]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data=avg_weekly_sales_per_month(clean_data)
agg_data

,Month,Avg_Sales
0,1,33160.0
1,2,34342.0
2,3,33227.0
3,4,33415.0
4,5,33340.0
5,6,34582.0
6,7,33931.0
7,8,33645.0
8,9,33267.0
9,10,32737.0


In [250]:
agg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Month      12 non-null     int64  
 1   Avg_Sales  12 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 320.0 bytes


In [251]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path,index=False)
    agg_data.to_csv(agg_data_file_path,index=False)
    pass

In [252]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths 
load(clean_data,'clean_data.csv',agg_data,'agg_data.csv')

In [253]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    return os.path.exists(file_path)
    

In [254]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation('clean_data.csv')

True

In [255]:
validation('agg_data.csv')

True